In [ ]:
# Databricks notebook source


# Exercice 03 : Fonctions de Fenetre (Window Functions)

## Objectifs pedagogiques

A la fin de cet exercice, vous serez capable de :
- Comprendre le concept de fenetre (window)
- Utiliser les fonctions de classement (rank, dense_rank, row_number)
- Calculer des sommes cumulatives et moyennes mobiles
- Utiliser lag() et lead() pour comparer avec les lignes precedentes/suivantes
- Partitionner les donnees pour des calculs par groupe

## Duree estimee : 75 minutes



## PARTIE 1 : Concepts theoriques

### Qu'est-ce qu'une Window Function ?

Les fonctions de fenetre effectuent des calculs sur un ensemble de lignes
qui sont liees a la ligne courante, SANS regrouper les lignes (contrairement a GROUP BY).

**Difference cle avec GROUP BY** :
- GROUP BY : Reduit le nombre de lignes (une ligne par groupe)
- Window Function : Conserve toutes les lignes (ajoute des colonnes calculees)

### Structure d'une Window

Une fenetre se definit par :
1. **PARTITION BY** : Diviser les donnees en groupes
2. **ORDER BY** : Trier les lignes dans chaque partition
3. **FRAME** : Definir la plage de lignes a considerer (optionnel)

### Categories de fonctions :

1. **Ranking** : row_number(), rank(), dense_rank(), percent_rank()
2. **Analytiques** : lag(), lead(), first(), last()
3. **Agregats** : sum(), avg(), min(), max(), count()



## PARTIE 2 : Preparation des donnees


In [ ]:

print("PREPARATION : Creer un dataset enrichi")
print("=" * 70)

from pyspark.sql.functions import col, when, length

# Charger et enrichir les donnees
df_gares = spark.table("gares_silver")

# Creer un dataset avec plus d'informations pour les exemples
df_enrichi = df_gares \
    .withColumn(
        "region",
        when(col("code_departement").isin("75", "77", "78", "91", "92", "93", "94", "95"), "Ile-de-France")
        .when(col("code_departement").isin("69", "01", "42", "38", "73", "74"), "Auvergne-Rhone-Alpes")
        .when(col("code_departement").isin("13", "83", "84", "04", "05", "06"), "PACA")
        .when(col("code_departement").isin("59", "62", "02", "80", "60"), "Hauts-de-France")
        .when(col("code_departement").isin("33", "40", "47", "64"), "Nouvelle-Aquitaine")
        .otherwise("Autre")
    ) \
    .withColumn("longueur_nom", length(col("nom")))

print(f"Dataset enrichi : {df_enrichi.count()} lignes")
df_enrichi.select("nom", "code_departement", "region", "segment_clean").show(10)



## PARTIE 3 : Fonctions de classement


In [ ]:

print("EXERCICE 3.1 : ROW_NUMBER, RANK et DENSE_RANK")
print("=" * 70)

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, col

# Definir une fenetre : partitionner par region, trier par nom
window_region = Window.partitionBy("region").orderBy("nom")

# Appliquer les trois fonctions de classement
df_classement = df_enrichi \
    .filter(col("region").isin("Ile-de-France", "PACA")) \
    .withColumn("row_number", row_number().over(window_region)) \
    .withColumn("rank", rank().over(window_region)) \
    .withColumn("dense_rank", dense_rank().over(window_region)) \
    .select("region", "nom", "row_number", "rank", "dense_rank")

print("Comparaison des fonctions de classement :")
df_classement.orderBy("region", "row_number").show(20)

print("""
DIFFERENCES :
- row_number() : Numerotation continue (1, 2, 3, 4...)
- rank() : Saute des rangs en cas d'egalite (1, 2, 2, 4...)
- dense_rank() : Pas de saut (1, 2, 2, 3...)
""")



## EXERCICE PRATIQUE 3.2

Pour chaque departement, classez les gares par longueur de nom (decroissant) :
1. Utilisez row_number()
2. Affichez uniquement : code_departement, nom, longueur_nom, rang
3. Filtrez pour garder les 3 premieres de chaque departement
4. Limitez aux departements 75, 69, 13


In [ ]:

# A COMPLETER
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

# Definir la fenetre
window_dept = Window.partitionBy("code_departement").orderBy(col("longueur_nom").desc())

# Appliquer row_number et filtrer
df_top3_noms_longs = df_enrichi \
    .filter(col("code_departement").isin("75", "69", "13")) \
    .withColumn("rang", row_number().over(window_dept)) \
    .filter(col("rang") <= 3) \
    .select("code_departement", "nom", "longueur_nom", "rang") \
    .orderBy("code_departement", "rang")

print("Top 3 des gares avec les noms les plus longs par departement :")
df_top3_noms_longs.show(20, truncate=False)



## PARTIE 4 : LAG et LEAD (valeurs precedentes/suivantes)


In [ ]:

print("EXERCICE 3.3 : LAG et LEAD")
print("=" * 70)

from pyspark.sql.functions import lag, lead, col

# Fenetre : trier les gares d'Ile-de-France par nom
window_idf = Window.partitionBy("region").orderBy("nom")

df_lag_lead = df_enrichi \
    .filter(col("region") == "Ile-de-France") \
    .withColumn("gare_precedente", lag("nom", 1).over(window_idf)) \
    .withColumn("gare_suivante", lead("nom", 1).over(window_idf)) \
    .select("nom", "gare_precedente", "gare_suivante")

print("LAG (precedent) et LEAD (suivant) :")
df_lag_lead.show(15, truncate=False)



## EXERCICE PRATIQUE 3.4

Pour les gares du departement 75, triees par latitude (sud vers nord) :
1. Affichez la gare precedente et suivante
2. Calculez la difference de latitude avec la gare precedente
3. Affichez : nom, latitude, gare_precedente, diff_latitude


In [ ]:

# A COMPLETER
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, round

# Fenetre : trier par latitude
window_lat = Window.partitionBy("code_departement").orderBy("latitude")

df_paris_lat = df_enrichi \
    .filter(col("code_departement") == "75") \
    .withColumn("gare_precedente", lag("nom", 1).over(window_lat)) \
    .withColumn("lat_precedente", lag("latitude", 1).over(window_lat)) \
    .withColumn("diff_latitude", round(col("latitude") - col("lat_precedente"), 6)) \
    .select("nom", "latitude", "gare_precedente", "diff_latitude") \
    .orderBy("latitude")

print("Gares de Paris triees du sud au nord :")
df_paris_lat.show(20, truncate=False)



## PARTIE 5 : Agregations dans une fenetre


In [ ]:

print("EXERCICE 3.5 : Somme cumulative (Running Total)")
print("=" * 70)

from pyspark.sql.window import Window
from pyspark.sql.functions import sum, count, col

# Compter les gares cumulees par region (ordre alphabetique)
window_cumul = Window.partitionBy("region").orderBy("nom").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_cumul = df_enrichi \
    .filter(col("region").isin("Ile-de-France", "PACA")) \
    .withColumn("numero", row_number().over(Window.partitionBy("region").orderBy("nom"))) \
    .withColumn("cumul_gares", count("*").over(window_cumul)) \
    .select("region", "nom", "numero", "cumul_gares")

print("Cumul du nombre de gares par region :")
df_cumul.show(30)



## EXERCICE PRATIQUE 3.6

Pour chaque region, calculez :
1. Le nombre cumule de gares (ordre alphabetique)
2. Le pourcentage du total de la region
3. Affichez : region, nom, cumul, pourcentage


In [ ]:

# A COMPLETER
from pyspark.sql.window import Window
from pyspark.sql.functions import count, col, round

# Fenetre pour le cumul
window_cumul = Window.partitionBy("region").orderBy("nom").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Fenetre pour le total
window_total = Window.partitionBy("region")

df_progression = df_enrichi \
    .filter(col("region").isin("Ile-de-France", "PACA", "Hauts-de-France")) \
    .withColumn("cumul", count("*").over(window_cumul)) \
    .withColumn("total", count("*").over(window_total)) \
    .withColumn("pourcentage", round((col("cumul") * 100.0 / col("total")), 2)) \
    .select("region", "nom", "cumul", "total", "pourcentage")

print("Progression cumulee par region :")
df_progression.show(30)



## PARTIE 6 : Moyenne mobile (Moving Average)


In [ ]:

print("EXERCICE 3.7 : Moyenne mobile")
print("=" * 70)

from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, round, row_number

# Fenetre glissante : 2 lignes avant + ligne courante + 2 lignes apres
window_mobile = Window.partitionBy("region").orderBy("latitude").rowsBetween(-2, 2)

df_mobile = df_enrichi \
    .filter(col("region") == "Ile-de-France") \
    .withColumn("lat_moyenne_mobile", round(avg("latitude").over(window_mobile), 6)) \
    .withColumn("lon_moyenne_mobile", round(avg("longitude").over(window_mobile), 6)) \
    .select("nom", "latitude", "lat_moyenne_mobile", "longitude", "lon_moyenne_mobile") \
    .orderBy("latitude")

print("Moyenne mobile sur 5 gares (2 avant, courante, 2 apres) :")
df_mobile.show(20)



## PARTIE 7 : FIRST et LAST


In [ ]:

print("EXERCICE 3.8 : Premiere et derniere valeur de la fenetre")
print("=" * 70)

from pyspark.sql.functions import first, last, col

# Fenetre : toute la partition
window_full = Window.partitionBy("region").orderBy("nom").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_first_last = df_enrichi \
    .filter(col("region").isin("Ile-de-France", "PACA")) \
    .withColumn("premiere_gare_region", first("nom").over(window_full)) \
    .withColumn("derniere_gare_region", last("nom").over(window_full)) \
    .select("region", "nom", "premiere_gare_region", "derniere_gare_region") \
    .distinct()

print("Premiere et derniere gare de chaque region (ordre alphabetique) :")
df_first_last.show(10, truncate=False)



## EXERCICE PRATIQUE 3.9 : Comparaison avec extremes

Pour chaque region :
1. Trouvez la gare la plus au nord (max latitude)
2. Trouvez la gare la plus au sud (min latitude)
3. Pour chaque gare, calculez sa distance (en latitude) par rapport au nord et au sud
4. Limitez a la region Ile-de-France


In [ ]:

# A COMPLETER
from pyspark.sql.functions import max, min, col, round

# Fenetre pour toute la region
window_region_full = Window.partitionBy("region")

df_distances = df_enrichi \
    .filter(col("region") == "Ile-de-France") \
    .withColumn("lat_max_region", max("latitude").over(window_region_full)) \
    .withColumn("lat_min_region", min("latitude").over(window_region_full)) \
    .withColumn("distance_du_nord", round(col("lat_max_region") - col("latitude"), 6)) \
    .withColumn("distance_du_sud", round(col("latitude") - col("lat_min_region"), 6)) \
    .select("nom", "latitude", "lat_max_region", "lat_min_region", "distance_du_nord", "distance_du_sud") \
    .orderBy("latitude")

print("Distance de chaque gare par rapport aux extremes de la region :")
df_distances.show(20)



## PARTIE 9 : Percentiles et quartiles


In [ ]:

print("EXERCICE 3.11 : Calcul de percentiles")
print("=" * 70)

from pyspark.sql.functions import percent_rank, ntile, col, round

# Fenetre : trier par longueur de nom
window_percentile = Window.partitionBy("region").orderBy("longueur_nom")

df_percentiles = df_enrichi \
    .filter(col("region").isin("Ile-de-France", "PACA")) \
    .withColumn("percent_rank", round(percent_rank().over(window_percentile), 4)) \
    .withColumn("quartile", ntile(4).over(window_percentile)) \
    .withColumn("decile", ntile(10).over(window_percentile)) \
    .select("region", "nom", "longueur_nom", "percent_rank", "quartile", "decile")

print("Percentiles et quartiles basés sur la longueur du nom :")
df_percentiles.orderBy("region", "longueur_nom").show(30)

print("""
EXPLICATION :
- percent_rank() : Position relative (0 a 1) dans la partition
- ntile(4) : Divise en 4 quartiles egaux
- ntile(10) : Divise en 10 deciles egaux
""")



## EXERCICE FINAL 3.12 : Analyse complete avec Window Functions

Creez une analyse qui, pour chaque region :

1. Classe les gares par latitude (du sud au nord)
2. Calcule la latitude moyenne mobile sur 3 gares (1 avant, courante, 1 apres)
3. Indique le nom de la gare precedente et suivante
4. Calcule le rang dans la region
5. Indique dans quel quartile se trouve la gare (ntile)
6. Limite aux regions : Ile-de-France, PACA, Hauts-de-France
7. Sauvegarde le resultat dans une table


In [ ]:

# A COMPLETER - SOLUTION PROPOSEE
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, avg, lag, lead, ntile, col, round

# Fenetres
window_lat = Window.partitionBy("region").orderBy("latitude")
window_mobile = Window.partitionBy("region").orderBy("latitude").rowsBetween(-1, 1)

# Pipeline complete
df_analyse_window = df_enrichi \
    .filter(col("region").isin("Ile-de-France", "PACA", "Hauts-de-France")) \
    .withColumn("rang_sud_nord", row_number().over(window_lat)) \
    .withColumn("lat_moyenne_mobile", round(avg("latitude").over(window_mobile), 6)) \
    .withColumn("gare_precedente", lag("nom", 1).over(window_lat)) \
    .withColumn("gare_suivante", lead("nom", 1).over(window_lat)) \
    .withColumn("quartile", ntile(4).over(window_lat)) \
    .select(
        "region",
        "nom",
        "latitude",
        "rang_sud_nord",
        "lat_moyenne_mobile",
        "gare_precedente",
        "gare_suivante",
        "quartile"
    ) \
    .orderBy("region", "latitude")

print("Analyse complete avec window functions :")
df_analyse_window.show(30, truncate=False)

# Sauvegarder
df_analyse_window.write.format("delta").mode("overwrite").saveAsTable("analyse_window_gares")
print("\nTable 'analyse_window_gares' creee !")



## PARTIE 10 : Window Functions en SQL


In [ ]:

%sql
-- EXERCICE 3.13 : Meme analyse en SQL

SELECT
code_departement,
nom,
latitude,
ROW_NUMBER() OVER (PARTITION BY code_departement ORDER BY latitude) as rang,
RANK() OVER (PARTITION BY code_departement ORDER BY latitude) as rank,
LAG(nom, 1) OVER (PARTITION BY code_departement ORDER BY latitude) as gare_precedente,
LEAD(nom, 1) OVER (PARTITION BY code_departement ORDER BY latitude) as gare_suivante
FROM gares_silver
WHERE code_departement IN ('75', '69', '13')
ORDER BY code_departement, latitude
LIMIT 30


In [ ]:

%sql
-- Moyenne mobile en SQL
SELECT
code_departement,
nom,
latitude,
ROUND(AVG(latitude) OVER (
PARTITION BY code_departement
ORDER BY latitude
ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
), 6) as lat_moyenne_mobile
FROM gares_silver
WHERE code_departement = '75'
ORDER BY latitude



## RESUME DES CONCEPTS APPRIS

Dans cet exercice, vous avez appris :

1. **Concept de fenetre** : PARTITION BY, ORDER BY, FRAME
2. **Classement** : `row_number()`, `rank()`, `dense_rank()`, `percent_rank()`, `ntile()`
3. **Acces aux lignes** : `lag()`, `lead()`, `first()`, `last()`
4. **Agregations** : `sum()`, `avg()`, `count()`, `min()`, `max()` sur fenetre
5. **Frames** : `rowsBetween()`, `rangeBetween()`
6. **Cumuls** : Running totals, moyennes mobiles
7. **Percentiles** : `percent_rank()`, `ntile()`
8. **SQL** : Syntaxe `OVER (PARTITION BY... ORDER BY...)`

## Cas d'usage pratiques

- Classements et top N par categorie
- Comparaisons avec periode precedente
- Calcul de tendances (moyennes mobiles)
- Detection d'anomalies (ecart par rapport a la moyenne)
- Segmentation (quartiles, deciles)

## Prochaine etape

Passez a l'**Exercice 04 : Delta Lake avance** pour apprendre :
- Time Travel (voyage dans le temps)
- MERGE (upserts)
- Change Data Capture (CDC)
- Optimisations Delta (OPTIMIZE, Z-ORDER, VACUUM)
